In [1]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from datetime import date

In [2]:
from sklearn import preprocessing


In [3]:
import seaborn as sns

In [4]:
from sklearn.model_selection import KFold

In [5]:
from sklearn.metrics import mean_absolute_error

In [6]:
from sklearn.metrics import mean_squared_error

In [7]:
# Load Train Data
train = pd.read_csv('../data/processed/train_norm.csv', sep=';')

In [8]:
train.shape

(34540, 36)

In [9]:
train['fecha_venta_norm'] = pd.to_datetime(train['fecha_venta_norm'])

In [10]:
train['fecha_venta_norm'] = train['fecha_venta_norm'].dt.date

In [11]:
train['year'] = pd.to_datetime(train.fecha_venta_norm).dt.year
train['month'] = pd.to_datetime(train.fecha_venta_norm).dt.month

In [12]:
train.fecha_venta_norm.unique()

array([datetime.date(2012, 12, 1), datetime.date(2013, 12, 1),
       datetime.date(2013, 11, 1), datetime.date(2012, 11, 1),
       datetime.date(2014, 11, 1), datetime.date(2014, 10, 1),
       datetime.date(2013, 10, 1), datetime.date(2013, 8, 1),
       datetime.date(2012, 10, 1), datetime.date(2014, 9, 1),
       datetime.date(2013, 9, 1), datetime.date(2013, 6, 1),
       datetime.date(2013, 7, 1)], dtype=object)

In [13]:
# Filtramos los meses que consideramos buenos para el entrenamiento (11 y 12)
train = train[train.fecha_venta_norm.isin([ date(2012, 11, 1),
                                                 date(2012, 12, 1),
                                                date(2013, 11, 1), 
                                                 date(2013, 12, 1), 
                                                 date(2014, 11, 1)
])]

In [14]:
train.shape

(33960, 38)

In [15]:
predictors = ['id_pos',
              'unidades', 'canal', 'competidores',
       'ingreso_mediana', 'densidad_poblacional',
       'pct_0a5', 'pct_5a9', 'pct_10a14', 'pct_15a19', 'pct_20a24',
       'pct_25a29', 'pct_30a34', 'pct_35a39', 'pct_40a44', 'pct_45a49',
       'pct_50a54', 'pct_55a59', 'pct_60a64', 'pct_65a69', 'pct_70a74',
       'pct_75a79', 'pct_80a84', 'pct_85ainf', 'pct_bachelors',
       'pct_doctorados', 'pct_secundario', 'pct_master', 'pct_bicicleta',
       'pct_omnibus', 'pct_subtes', 'pct_taxi', 'pct_caminata',
       'mediana_valor_hogar']

In [16]:
train = train[predictors]

In [24]:
X = train[predictors].copy()

In [26]:
# definimos los extremos de los valores con los correspondiente a Q1 y Q3
y = train['unidades'].clip(4,23)

In [28]:
X.drop('unidades', inplace=True, axis=1)

In [29]:
# fill Nans
for c in X.columns[X.dtypes != 'object']:
    X[c].fillna(X[c].median(), inplace=True)

In [30]:
X.columns[X.dtypes == 'object']

Index(['canal'], dtype='object')

In [31]:
# Dummy
canal_dummy = pd.get_dummies(X['canal'])

In [32]:
X = pd.merge(X, canal_dummy, how='inner', left_index=True, right_index=True)

In [33]:
X.drop('canal', axis=1, inplace=True)

In [34]:
X = sm.add_constant(X)

/opt/anaconda3/envs/signalenv/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [35]:
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               unidades   R-squared:                       0.077
Model:                            OLS   Adj. R-squared:                  0.076
Method:                 Least Squares   F-statistic:                     81.20
Date:                Fri, 28 Jun 2019   Prob (F-statistic):               0.00
Time:                        03:34:55   Log-Likelihood:            -1.1616e+05
No. Observations:               33960   AIC:                         2.324e+05
Df Residuals:                   33924   BIC:                         2.327e+05
Df Model:                          35                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    8.9459      0.999      8.955      0.000       6.988      10.904
id_pos                  -0.0004   7.51e-06    -51.838      0.000      -0.000      -0.000
competidores             0.0234      0.049      0.481      0.630      -0.072       0.119
ingreso_mediana         -0.0738      0.096     -0.771      0.441      -0.261       0.114
densidad_poblacional    -0.0320      0.084     -0.380      0.704      -0.197       0.133
pct_0a5                  0.0470      0.086      0.548      0.584      -0.121       0.215
pct_5a9                 -0.1016      0.110     -0.922      0.357      -0.318       0.115
pct_10a14                0.1152      0.099      1.163      0.245      -0.079       0.309
pct_15a19                0.0228      0.076      0.302      0.763      -0.125       0.171
pct_20a24               -0.0865      0.094     -0.923      0.356      -0.270       0.097
pct_25a29                0.0845      0.106      0.799      0.424      -0.123       0.292
pct_30a34                0.0815      0.107      0.762      0.446      -0.128       0.291
pct_35a39               -0.0644      0.085     -0.760      0.447      -0.231       0.102
pct_40a44               -0.0737      0.078     -0.948      0.343      -0.226       0.079
pct_45a49                0.0969      0.082      1.188      0.235      -0.063       0.257
pct_50a54               -0.0404      0.083     -0.489      0.625      -0.202       0.122
pct_55a59               -0.0092      0.089     -0.103      0.918      -0.184       0.166
pct_60a64                0.1105      0.100      1.105      0.269      -0.086       0.306
pct_65a69               -0.1255      0.112     -1.117      0.264      -0.346       0.095
pct_70a74                0.1444      0.118      1.220      0.222      -0.088       0.376
pct_75a79                0.0033      0.121      0.027      0.978      -0.235       0.241
pct_80a84               -0.1606      0.121     -1.328      0.184      -0.398       0.076
pct_85ainf               0.0942      0.085      1.106      0.269      -0.073       0.261
pct_bachelors            0.0905      0.105      0.860      0.390      -0.116       0.297
pct_doctorados           0.0031      0.073      0.043      0.965      -0.139       0.145
pct_secundario          -0.0109      0.086     -0.127      0.899      -0.180       0.158
pct_master               0.0922      0.106      0.873      0.383      -0.115       0.299
pct_bicicleta           -0.0197      0.052     -0.378      0.706      -0.122       0.083
pct_omnibus              0.0015      0.058      0.025      0.980      -0.113       0.116
pct_subtes               0.0321      0.064      0.505      0.613      -0.092       0.157
pct_taxi                 0.0263      0.048      0.543      0.587      -0.069       0.121
pct_caminata            -0.0587      0.060     -0.977    

In [26]:
predictors = [ 'id_pos', 
       'ingreso_mediana', 'densidad_poblacional',
        'pct_5a9', 'pct_10a14', 'pct_15a19', 'pct_20a24',
        'pct_30a34',  'pct_40a44', 'pct_45a49',
       'pct_50a54',  'pct_60a64', 'pct_65a69', 'pct_70a74',
       'pct_75a79', 'pct_80a84', 'pct_85ainf', 
         'pct_master', 
        'pct_subtes',  'pct_caminata',
       'mediana_valor_hogar' ,'ALMACEN', 'MAXIKIOSCO'
    
]

In [27]:
X_sel = X[predictors]
X_sel = sm.add_constant(X_sel)

In [28]:
model = sm.OLS(y, X_sel).fit()
predictions = model.predict(X_sel) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               unidades   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     143.4
Date:                Sat, 22 Jun 2019   Prob (F-statistic):               0.00
Time:                        03:28:40   Log-Likelihood:            -1.3122e+05
No. Observations:               29168   AIC:                         2.625e+05
Df Residuals:                   29144   BIC:                         2.627e+05
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   11.9137      7.259      1.641      0.101      -2.314      26.142
id_pos                  -0.0013   2.35e-05    -56.422      0.000      -0.001      -0.001
ingreso_mediana         -0.1574      0.278     -0.565      0.572      -0.703       0.388
densidad_poblacional     0.0577      0.228      0.253      0.800      -0.389       0.505
pct_5a9                 -0.4256      0.291     -1.461      0.144      -0.997       0.145
pct_10a14                0.2411      0.312      0.774      0.439      -0.370       0.852
pct_15a19                0.2667      0.224      1.193      0.233      -0.172       0.705
pct_20a24               -0.3753      0.276     -1.360      0.174      -0.916       0.165
pct_30a34                0.3633      0.254      1.433      0.152      -0.134       0.860
pct_40a44               -0.2766      0.221     -1.253      0.210      -0.709       0.156
pct_45a49                0.1978      0.255      0.776      0.438      -0.302       0.698
pct_50a54               -0.2522      0.233     -1.081      0.280      -0.709       0.205
pct_60a64                0.3292      0.280      1.178      0.239      -0.219       0.877
pct_65a69               -0.4529      0.355     -1.277      0.202      -1.148       0.242
pct_70a74                0.6015      0.370      1.625      0.104      -0.124       1.327
pct_75a79                0.3684      0.383      0.963      0.336      -0.382       1.119
pct_80a84               -0.7331      0.383     -1.914      0.056      -1.484       0.018
pct_85ainf               0.2497      0.269      0.929      0.353      -0.277       0.777
pct_master               0.4416      0.230      1.924      0.054      -0.008       0.891
pct_subtes               0.1512      0.193      0.782      0.434      -0.228       0.530
pct_caminata            -0.4408      0.178     -2.476      0.013      -0.790      -0.092
mediana_valor_hogar     -0.3378      0.250     -1.349      0.177      -0.828       0.153
ALMACEN                 20.6133      7.260      2.839      0.005       6.383      34.844
MAXIKIOSCO               6.5239      7.403      0.881      0.378      -7.986      21.033
==============================================================================
Omnibus:                    12776.947   Durbin-Watson:                   2.302
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            73999.612
Skew:                           2.051   Prob(JB):                         0.00
Kurtosis:                       9.639   Cond. No.                     1.09e+06
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.09e+06. This might indicate that there are
strong multicollinearit

In [29]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               unidades   R-squared:                       0.102
Model:                            OLS   Adj. R-squared:                  0.101
Method:                 Least Squares   F-statistic:                     143.4
Date:                Sat, 22 Jun 2019   Prob (F-statistic):               0.00
Time:                        03:28:40   Log-Likelihood:            -1.3122e+05
No. Observations:               29168   AIC:                         2.625e+05
Df Residuals:                   29144   BIC:                         2.627e+05
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   11.9137 

#### Linear Regression

In [30]:
y = pd.DataFrame(y, index=X.index)

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X[predictors], y, test_size=0.3, random_state=47)

In [32]:
model = LinearRegression(n_jobs=4)
maes = []
kf = KFold(n_splits=3)
for train_index, test_index in kf.split(X_train):
    KX_train = X_train.iloc[train_index,:]
    KX_test = X_train.iloc[test_index,:]
    Ky_train = y_train.iloc[train_index,:]
    Ky_test =  y_train.iloc[test_index,:]
    #print(KX_train.isnull().any())
    #print(Ky_train)
    model.fit(KX_train, Ky_train)
    y_pred = model.predict(KX_test)
    maes.append(mean_absolute_error(Ky_test, y_pred))
    
    
    
    

In [33]:
maes

[15.5011443442705, 16.000053055593714, 15.605580167119715]

In [34]:
print("Mean of MAEs:",np.mean(maes))

Mean of MAEs: 15.702259188994644


In [35]:
print("SDV of MAEs:", np.std(maes))

SDV of MAEs: 0.21484506124424807
